In [58]:
import tweetnlp
model = tweetnlp.load_model('irony')

In [59]:
result = model.irony('Wow... soooo cool', return_probability=True)
type(result['probability']['irony'])
print(result)

{'label': 'irony', 'probability': {'non_irony': 0.03373716026544571, 'irony': 0.9662628173828125}}


In [60]:
# taken from gpt.ipynb file
def calcFScore(truepos, trueneg, falsepos, falseneg):
    FScoreResults = {}
    FScoreResults['accuracy'] = (truepos + trueneg)/(truepos + trueneg + falsepos + falseneg)
    FScoreResults['accuracy'] = (truepos + trueneg)/(truepos + trueneg + falsepos + falseneg)
    if (truepos + falsepos > 0):
        FScoreResults['precision'] = truepos/(truepos + falsepos)
    else:
        FScoreResults['precision'] = 0

    if(truepos + falseneg > 0):
        FScoreResults['recall'] = truepos/(truepos + falseneg)
    else:
        FScoreResults['recall'] = 0
        
    if((FScoreResults['precision'] + FScoreResults['recall']) > 0):
        FScoreResults['F1'] = (2 * FScoreResults['precision'] * FScoreResults['recall'])/(FScoreResults['precision'] + FScoreResults['recall'])
    else:
        FScoreResults['F1'] = 0
    return FScoreResults

In [61]:
# Calculate the scores based on the answers from pysentimiento, this method is simply adapted from the other scores methods for GPT
def scoresSentimiento(dataset, resultColumn):
    truepos = 0
    falsepos = 0
    trueneg = 0
    falseneg = 0

    for index, row in dataset.iterrows():
        sentimientoEval = row[resultColumn]
        if (int(row[1]) == 1):
            if (int(sentimientoEval) == 1):
                truepos = truepos + 1
            else:
                falseneg = falseneg + 1
        elif (int(row[1]) == 0):
            if (int(sentimientoEval) == 1):
                falsepos = falsepos + 1
            else:
                trueneg = trueneg + 1

    numResults = calcFScore(truepos, trueneg, falsepos, falseneg)
    numResults['tp'] = truepos
    numResults['fn'] = falseneg
    numResults['fp'] = falsepos
    numResults['tn'] = trueneg
    return numResults

In [67]:
import pandas as pd

datasetName = "fixedsetreadin"
datasetPath = "datasets\\" + datasetName + ".csv"
data = pd.read_csv(datasetPath)
data = data.head(100)

results = []
resultsPerc = []

for index, row in data.iterrows():
    result = model.irony(row[0], return_probability=True)
    resultsPerc.append(result['probability']['irony'])
    if(result['probability']['irony'] >= 0.5):
        results.append(1)
    else:
        results.append(0)

data['result'] = results
data['resultPercentage'] = resultsPerc
scores = scoresSentimiento(data, 2)
scores

linesToWrite = ['Results for TweetNLP using dataset ' + datasetName +  ': \n',
        'Confusion Matrix:' + '\n',
        str(scores['tp']) + '  ' + str(scores['fn']) + '\n',
        str(scores['fp']) + '  ' + str(scores['tn']) + '\n',
        'Accuracy: ' + str(scores['accuracy']) + '\n',
        'Precision: ' + str(scores['precision']) + '\n',
        'Recall: ' + str(scores['recall']) + '\n',
        'F1-Score: ' + str(scores['F1']) + '\n\n']

scores

#file = open("TweetNLP results" + "\\metadata.txt", "w")

#data.to_csv("TweetNLP results" + "\\results.csv", index=False)

#file.writelines(linesToWrite)
#file.close()

{'accuracy': 0.39,
 'precision': 0.2948717948717949,
 'recall': 0.7931034482758621,
 'F1': 0.42990654205607476,
 'tp': 23,
 'fn': 6,
 'fp': 55,
 'tn': 16}

In [68]:
import pandas as pd
# check if values are really or only mostly the same
datasetName = "fixedsetreadin"
datasetPath = "datasets\\" + datasetName + ".csv"
data = pd.read_csv(datasetPath)
data = data.head(100)

for i in range(10):
    results2 = []
    for index, row in data.iterrows():
        result = model.irony(row[0], return_probability=True)
        if(result['probability']['irony'] >= 0.5):
            results2.append(1)
        else:
            results2.append(0)

    data['result'] = results2
    scores2 = scoresSentimiento(data, 2)
    scores2
    print(scores == scores2)


True
True
True
True
True
True
True
True
True
True
